In [1]:
import pandas as pd
import glob

In [2]:
path = r"C:\Users\Andrey\Desktop\inline.ts"

In [15]:
df = pd.read_csv(path, sep=" ", skiprows=[i for i in range(13)], index_col=False, header=None
                 # columns=["Entity", "Num",],
                )

In [16]:
df_vrtx = df[df[0] == "VRTX"]
df_vrtx

,0,1,2,3,4
0,VRTX,0.0,540671.551676,7.677381e+06,-555.921134
1,VRTX,1.0,540666.095634,7.677320e+06,-822.839445
2,VRTX,2.0,540652.691286,7.677170e+06,-1130.265652
3,VRTX,3.0,540642.284392,7.677054e+06,-1241.804169
4,VRTX,4.0,540828.336281,7.677729e+06,-72.975860
...,...,...,...,...,...
79,VRTX,79.0,541692.744834,7.677567e+06,-305.116356
80,VRTX,80.0,541685.324866,7.677485e+06,-610.139689
81,VRTX,81.0,541679.363636,7.677418e+06,-928.191257
82,VRTX,82.0,541673.941274,7.677358e+06,-1230.351450


In [21]:
df_trgl = df[df[0] == "TRGL"].drop(columns=[0, 4])
df_trgl

,1,2,3
84,40.0,44.0,39.0
85,34.0,33.0,32.0
86,5.0,31.0,32.0
87,31.0,35.0,34.0
88,32.0,31.0,34.0
...,...,...,...
214,82.0,28.0,81.0
215,27.0,80.0,81.0
216,24.0,64.0,23.0
217,61.0,64.0,60.0


In [ ]:
for elem in df_trgl.iterrows():
    vert_inds = elem[1].tolist()
    vert_inds = [int(ind) for ind in vert_inds]
    print(len(list(df_vrtx.loc[vert_inds, 2:5].itertuples(index=False, name=None))))

In [8]:
import ezdxf
from ezdxf.render import MeshBuilder

doc = ezdxf.new("R2000")
msp = doc.modelspace()


files = glob.glob(r"C:\Users\Andrey\Desktop\Структурки\*ts")

for file in files:
    df = pd.read_csv(file, sep=" ", skiprows=[i for i in range(13)], index_col=False, header=None
                 # columns=["Entity", "Num",],
                )
    df_vrtx = df[df[0] == "VRTX"]
    # df_vrtx[4] *= -1
    df_trgl = df[df[0] == "TRGL"].drop(columns=[0, 4])
    name=file.split("\\")[-1][:-3]
    doc.layers.add(name=name)
    mesh = msp.add_mesh(dxfattribs={"layer": name})
    with mesh.edit_data() as mesh_data:
        for elem in df_trgl.iterrows():
            vert_inds = elem[1].tolist()
            vert_inds = [int(ind) for ind in vert_inds]
            mesh_data.add_face(list(df_vrtx.loc[vert_inds, 2:5].itertuples(index=False, name=None)))
        mesh_data.optimize()

doc.saveas("Структурные карты (Mesh).dxf")